#### Importando bibliotecas

In [1]:
from pymongo import MongoClient
import requests
import pandas as pd

#### Conectando ao MongoDB Atlas

In [ ]:
# Conectar ao MongoDB Atlas
client = MongoClient("")
# Seleciona o banco de dados
db = client[""]
# Seleciona a coleção
collection = db["pesquisa_industrial_mensal2"]

In [3]:
# Recuperar dados da coleção
data = list(collection.find())
data

[]

In [4]:
# Converter para DataFrame
df = pd.DataFrame(data)
print(df)

Empty DataFrame
Columns: []
Index: []


#### Consumindo API e enviando dados para o banco

In [5]:
# Endpoint da API do IBGE para a pesquisa industrial mensal período de 1985 à 2001
url = "https://servicodados.ibge.gov.br/api/v3/agregados/76/periodos/198501-200103/variaveis/25?localidades=N3[all]&classificacao=7[23]|24[70]"

# Realizar requisições e consumir os dados
response = requests.get(url)

# Verificar se a requisição foi bem-sucedida
if response.status_code == 200:
    data = response.json()
    print("Dados da API extraídos com sucesso!")

    # Extrair e organizar os dados
    industrial_data = []
    for resultado in data:
        for serie in resultado["resultados"][0]["series"]:
            estado_nome = serie["localidade"]["nome"]  # Obtém o nome do estado
            codigo_localidade = serie["localidade"]["id"]  # Obtém o código da localidade
            for periodo, valor in serie["serie"].items():
                ano, mes = periodo[:4], periodo[4:]  # Separa ano e mês
                industrial_data.append({
                    "estado": estado_nome,
                    "codigo_estado": codigo_localidade,
                    "ano": ano,
                    "mes": mes,
                    "volume_servicos_industria": valor
                })
else:
    print("Erro ao acessar API:", response.status_code, response.text)


Dados da API extraídos com sucesso!


In [6]:
# Converter para DataFrame
df = pd.DataFrame(industrial_data)
df.head(585)

,estado,codigo_estado,ano,mes,volume_servicos_industria
0,Minas Gerais,31,1985,01,53.92
1,Minas Gerais,31,1985,02,53.77
2,Minas Gerais,31,1985,03,66.30
3,Minas Gerais,31,1985,04,69.43
4,Minas Gerais,31,1985,05,81.28
...,...,...,...,...,...
580,São Paulo,35,2000,11,117901.00
581,São Paulo,35,2000,12,113460.00
582,São Paulo,35,2001,01,108108.00
583,São Paulo,35,2001,02,110787.00


In [7]:
#tranformar dados em dicionário para inserir no mongo
data_dict = df.to_dict("records")
print(data_dict)

[{'estado': 'Minas Gerais', 'codigo_estado': '31', 'ano': '1985', 'mes': '01', 'volume_servicos_industria': '53.92'}, {'estado': 'Minas Gerais', 'codigo_estado': '31', 'ano': '1985', 'mes': '02', 'volume_servicos_industria': '53.77'}, {'estado': 'Minas Gerais', 'codigo_estado': '31', 'ano': '1985', 'mes': '03', 'volume_servicos_industria': '66.30'}, {'estado': 'Minas Gerais', 'codigo_estado': '31', 'ano': '1985', 'mes': '04', 'volume_servicos_industria': '69.43'}, {'estado': 'Minas Gerais', 'codigo_estado': '31', 'ano': '1985', 'mes': '05', 'volume_servicos_industria': '81.28'}, {'estado': 'Minas Gerais', 'codigo_estado': '31', 'ano': '1985', 'mes': '06', 'volume_servicos_industria': '85.59'}, {'estado': 'Minas Gerais', 'codigo_estado': '31', 'ano': '1985', 'mes': '07', 'volume_servicos_industria': '105.95'}, {'estado': 'Minas Gerais', 'codigo_estado': '31', 'ano': '1985', 'mes': '08', 'volume_servicos_industria': '117.18'}, {'estado': 'Minas Gerais', 'codigo_estado': '31', 'ano': '198

In [8]:
# Inserindo os dados no MongoDB
collection.insert_many(data_dict)

InsertManyResult([ObjectId('675943ee44a9c475eb06f38e'), ObjectId('675943ee44a9c475eb06f38f'), ObjectId('675943ee44a9c475eb06f390'), ObjectId('675943ee44a9c475eb06f391'), ObjectId('675943ee44a9c475eb06f392'), ObjectId('675943ee44a9c475eb06f393'), ObjectId('675943ee44a9c475eb06f394'), ObjectId('675943ee44a9c475eb06f395'), ObjectId('675943ee44a9c475eb06f396'), ObjectId('675943ee44a9c475eb06f397'), ObjectId('675943ee44a9c475eb06f398'), ObjectId('675943ee44a9c475eb06f399'), ObjectId('675943ee44a9c475eb06f39a'), ObjectId('675943ee44a9c475eb06f39b'), ObjectId('675943ee44a9c475eb06f39c'), ObjectId('675943ee44a9c475eb06f39d'), ObjectId('675943ee44a9c475eb06f39e'), ObjectId('675943ee44a9c475eb06f39f'), ObjectId('675943ee44a9c475eb06f3a0'), ObjectId('675943ee44a9c475eb06f3a1'), ObjectId('675943ee44a9c475eb06f3a2'), ObjectId('675943ee44a9c475eb06f3a3'), ObjectId('675943ee44a9c475eb06f3a4'), ObjectId('675943ee44a9c475eb06f3a5'), ObjectId('675943ee44a9c475eb06f3a6'), ObjectId('675943ee44a9c475eb06f3